# Introduction - Oil Economy and Production

The price of crude oil is largely controlled by the [United States' economy](https://www.investopedia.com/articles/investing/081315/opec-vs-us-who-controls-oil-prices.asp) due to the massive demand and [oil production](https://www.eia.gov/todayinenergy/detail.php?id=15631) in the United States. Most of the crude oil is sourced from oil rigs drilling, which can be tracked through [various sources](https://ycharts.com/indicators/reports/baker_hughes_rotary_rig_count). Because rig count is a good indication of how much crude oil would be produced, theoretically there should be a positive relationship between the two. However, the time it takes to obtain crude oil is lengthy; as a result the oil price should see a trending lag behind rig count. This analysis will utilize `plotly` and additional interactivity to examine the number of rigs and the oil price and determine the lag among the two. 


In [38]:
# import libraries

import pandas as pd
import pandas_datareader as pdr
import numpy as np
import plotly as py
import plotly.graph_objects as go
from pandas.tseries.offsets import MonthEnd
from ipywidgets import IntSlider, interact, interactive

### Disclaimer
As of May 2020, nbviewer does not support interative widget. Download the `ipynb` file along with the necessary `csv` files to unlock the slider interactive feature. 

## Crude Oil Data
First the price of oil is needed. Yahoo Finance is an excellent source to find stock data or benchmark data, and ```pandas_datareader``` is a great tool to extract that data. 



In [39]:
# pull the price of oil 
df_oil = pdr.DataReader('CL=F', data_source='yahoo', start='2000-01-01', end='2020-03-30').reset_index(drop=False)

# get the size of the dataframe
print(f'Dimension of data: {df_oil.shape}')

# take a look at the beginning and the end of the dataframe
print('\nBeginning of data\n')
print(df_oil.head())
print('\n\nEnd of data\n')
print(df_oil.tail())

Dimension of data: (5001, 7)

Beginning of data

        Date       High        Low   Open      Close   Volume  Adj Close
0 2000-03-22  28.250000  27.250000  27.65  27.459999  92302.0  27.459999
1 2000-03-23  27.780001  27.160000  27.65  27.309999  79373.0  27.309999
2 2000-03-24  28.150000  27.549999  27.85  27.980000  55693.0  27.980000
3 2000-03-27  28.020000  27.410000  27.65  27.730000  59199.0  27.730000
4 2000-03-28  27.870001  27.000000  27.60  27.080000  39487.0  27.080000


End of data

           Date       High        Low       Open      Close       Volume  \
4996 2020-03-26  24.219999  22.379999  24.170000  23.000000  269457259.0   
4997 2020-03-27  23.139999  20.879999  23.080000  21.840000  249623140.0   
4998 2020-03-29  20.930000  19.920000  20.930000  20.350000    3163583.0   
4999 2020-03-30  20.860001  19.270000  20.490000  20.459999  329616292.0   
5000 2020-03-31  21.889999  20.010000  21.290001  20.240000  290761366.0   

      Adj Close  
4996  23.000000  
4997 

### Oil price dataframe observation
Before proceeding, the data should be checked to ensure there is no potential missing data points between 2000 - 2020. 

There are 5003 rows of data available between 2000 and 2020. 

There are 52 weekends in a year, therefore 104 days. 

There are 10 federal holidays in a year that the stock markets are not open. 

Sanity check:

```( 365.25 [days/year] - (104 + 10) [days/year] ) * (2020 - 2000) [years] = 5025 days```

The expected number of days is greater than the data extracted. The inconsistency was investigated after splitting the csv file into many csv files separated by individual years. The data was examined, and the missing data was explained by days when the market closed early. Yahoo Finance did not have the data if the market was closed early. 

Overall, the expected data amount was within close proxmity to the actual data retrieved. 

## Rig Count Data
The rig count data was gathered from [Baker Hughes's website](https://bakerhughesrigcount.gcs-web.com/intl-rig-count?c=79687&p=irol-rigcountsintl) and imported.

`Worldwide Rig Counts` seems interesting because it includes all of the data from different regions of the world. 

`International Rig Counts`, on the other hand, only includes regions outside of the North America.

In [40]:
df_rig = pd.read_csv('Worldwide Rig Count Mar 2020.csv')
df_rig.head(20)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WORLDWIDE RIG COUNT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,2020,Latin America,Europe,Africa,Middle East,Asia Pacific,Total Intl.,Canada,U.S.,Total World,NaN
6,NaN,Jan,179,133,114,430,222,1078,204,791,2073,NaN
7,NaN,Feb,184,130,120,427,224,1085,249,791,2125,NaN
8,NaN,Mar,169,123,108,428,231,1059,133,772,1964,NaN
9,NaN,Apr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Cleaning
The data looks messy and chaotic, but cleaning is doable. 

The csv contained three variables that made it similar to a 3D-matrix: region, year, and month. Luckily, year and month could be combined into a date, ultimately the dataset could be boiled down to a 2D-matrix.

Before that, let's remove the 0th and 11th unnamed column, as well as remove all null data.

In [41]:
# remove 0th and 11th column
df_rig = df_rig.drop(columns=['Unnamed: 0', 'Unnamed: 11'])

# remove all NaN data
df_rig = df_rig.dropna()

# take a look at what the data looks like after moving NaN
print(df_rig.head(20))

   Unnamed: 1     Unnamed: 2 Unnamed: 3 Unnamed: 4   Unnamed: 5    Unnamed: 6  \
5        2020  Latin America     Europe     Africa  Middle East  Asia Pacific   
6         Jan            179        133        114          430           222   
7         Feb            184        130        120          427           224   
8         Mar            169        123        108          428           231   
18       Avg.            177        129        114          428           226   
20       2019  Latin America     Europe     Africa  Middle East  Asia Pacific   
21        Jan            195         86        109          402           232   
22        Feb            182         94        113          398           240   
23        Mar            188         95        127          395           234   
24        Apr            190         98        126          412           236   
25        May            179        186        123          410           228   
26        Jun            189

### Note:
It seemed like the columns needed a new header, and the rows needed to be reindexed.


```Avg.``` should be dropped as well, since it didn't appear to be helpful or useful.

In [42]:
# abbreivate each regions of the world
df_rig.columns = ['Date', 'LATAM', 'EU', 'AFR', 'ME', 'AP', 'Total_Intl', 'CAN', 'US', 'Total_World']

# remove avg rows and resetting index
df_rig = df_rig[~df_rig['Date'].str.contains('Avg')].reset_index(drop=True)

print(df_rig.head(10))

   Date          LATAM      EU     AFR           ME            AP  \
0  2020  Latin America  Europe  Africa  Middle East  Asia Pacific   
1   Jan            179     133     114          430           222   
2   Feb            184     130     120          427           224   
3   Mar            169     123     108          428           231   
4  2019  Latin America  Europe  Africa  Middle East  Asia Pacific   
5   Jan            195      86     109          402           232   
6   Feb            182      94     113          398           240   
7   Mar            188      95     127          395           234   
8   Apr            190      98     126          412           236   
9   May            179     186     123          410           228   

    Total_Intl     CAN    US  Total_World  
0  Total Intl.  Canada  U.S.  Total World  
1         1078     204   791         2073  
2         1085     249   791         2125  
3         1059     133   772         1964  
4  Total Intl.  Cana

### Merging dates
Now merge years and months together to form a ```Date```

In [43]:
# current year + adjustment for header
year = 2021
months = {'Jan': '01', 'Feb': '02', 'Mar': '03', 
          'Apr': '04', 'May': '05', 'Jun': '06',
          'Jul': '07', 'Aug': '08', 'Sep': '09', 
          'Oct': '10', 'Nov': '11', 'Dec': '12'}

for i in range(len(df_rig['Date'])):
    
    # if the row is a year with headers, aka not-actual-data, replace it with NaN
    if df_rig['Date'][i].isdigit():
        df_rig['Date'][i] = np.nan
        year -= 1
    
    # replace month abbreviations with datetime format
    else: 
        df_rig['Date'][i] = f"{year}-{months[df_rig['Date'][i]]}"
        
        
# remove all NaN generated from the cleaning process
df_rig.dropna(inplace=True)

### Finalizing rig count data cleaning
Now the null values can simply be removed once again, and convert ```Date``` to the proper type.

However there is a complication: ```Date``` only contains ```year-month```, therefore ```day``` would be automatically added whenever ```to_datetime``` is applied. The data would then be incorrect since it is interpreted as the rig count of the 1st day of every month, when in reality the rig count is the count on the last day of every month. 


The first step is to convert ```Date``` from object to datetime, then attach ```MonthEnd``` to it - this concatenates the last day of the month to datetime.

In [44]:
# convert to datetime and attach monthend
df_rig['Date'] = pd.to_datetime(df_rig['Date'], format='%Y-%m') + MonthEnd(1)

# convert all values other than 'Date' to integers
df_rig.iloc[:, 1:] = df_rig.iloc[:, 1:].astype(int)
        
print(df_rig.head(10))

         Date  LATAM   EU  AFR   ME   AP  Total_Intl  CAN    US  Total_World
1  2020-01-31    179  133  114  430  222        1078  204   791         2073
2  2020-02-29    184  130  120  427  224        1085  249   791         2125
3  2020-03-31    169  123  108  428  231        1059  133   772         1964
5  2019-01-31    195   86  109  402  232        1024  176  1065         2265
6  2019-02-28    182   94  113  398  240        1027  230  1049         2306
7  2019-03-31    188   95  127  395  234        1039  151  1023         2213
8  2019-04-30    190   98  126  412  236        1062   66  1012         2140
9  2019-05-31    179  186  123  410  228        1126   70   986         2182
10 2019-06-30    189  193  116  413  227        1138  114   969         2221
11 2019-07-31    201  200  111  424  226        1162  121   955         2238


### Subsetting samples - taking what is needed
The original dataset started from 1975. The earliest finance data was in the year 2000, as a result, the rig count data needed to be subsetted.

In [45]:
# subsetting data to March 2000
df_rig20 = df_rig[ df_rig['Date'] >= '2000-03'].sort_values(by=['Date']).reset_index(drop=True)


print(df_rig20.head())

        Date  LATAM  EU  AFR   ME   AP  Total_Intl  CAN   US  Total_World
0 2000-03-31    199  82   45  149  131         606  394  773         1773
1 2000-04-30    199  79   42  153  129         602  146  805         1553
2 2000-05-31    215  76   46  156  134         627  189  844         1660
3 2000-06-30    225  90   46  159  137         657  291  878         1826
4 2000-07-31    233  85   45  160  139         662  308  942         1912


## Inflation
Transitioning back to the oil price, the oil price data spans over 20 years, therefore inflation would have be accounted for. 


The Consumer Price Index (CPI) can be found on [Federal Reserve Economic Data](https://fred.stlouisfed.org/series/CPIAUCNS). It has CPI since 1913.

In [46]:
# import cpi csv
inflation = pd.read_csv('CPIAUCNS.csv')

# convert 'DATE' to datetime
inflation['DATE'] = pd.to_datetime(inflation['DATE'])

# keep only data from year 2000 and on
inflation = inflation[inflation['DATE'] >= '2000'].reset_index(drop=True)

# convert inflation rate to multipliers using the last data point (Mar-2020) as the reference point
inflation['multiplier'] = inflation['CPIAUCNS'].iloc[-1] / inflation['CPIAUCNS']


print(inflation.tail())

          DATE  CPIAUCNS  multiplier
238 2019-11-01   257.208    1.003526
239 2019-12-01   256.974    1.004440
240 2020-01-01   257.971    1.000558
241 2020-02-01   258.678    0.997824
242 2020-03-01   258.115    1.000000


## Matching Game

Since inflation only had one data point for every month, whereas oil price had data on a daily basis, the months and years needed to match between the two datasets. Afterwards, the appropriate multiplier would be applied to the respective oil price. 

In [47]:
# extract month and year from the oil price, and inflation dataframe
df_oil['year'] = df_oil['Date'].dt.year
df_oil['month'] = df_oil['Date'].dt.month
inflation['year'] = inflation['DATE'].dt.year
inflation['month'] = inflation['DATE'].dt.month

# list to store values
adjusted = []

for i in range(len(df_oil)):
    adjusted.append(
        df_oil['Close'][i] * \
        inflation['multiplier'][(inflation['month'] == df_oil['month'][i]) \
                                & (inflation['year'] == df_oil['year'][i])].iloc[0])
    

# create new column with the adjusted price
df_oil['adjusted'] = pd.DataFrame(adjusted)

print(df_oil.head())

        Date       High        Low   Open      Close   Volume  Adj Close  \
0 2000-03-22  28.250000  27.250000  27.65  27.459999  92302.0  27.459999   
1 2000-03-23  27.780001  27.160000  27.65  27.309999  79373.0  27.309999   
2 2000-03-24  28.150000  27.549999  27.85  27.980000  55693.0  27.980000   
3 2000-03-27  28.020000  27.410000  27.65  27.730000  59199.0  27.730000   
4 2000-03-28  27.870001  27.000000  27.60  27.080000  39487.0  27.080000   

   year  month   adjusted  
0  2000      3  41.400921  
1  2000      3  41.174769  
2  2000      3  42.184916  
3  2000      3  41.807996  
4  2000      3  40.828003  


## Resampling

The current oil price data and rig count data are not the same size. The oil price curve needs to be smoothened in order for the two datasets to be the same size. There are many ways to smooth a curve, but resampling is one of the simplest methods while retaining lots of data information. 


Resampling a data set generates a unique sampling distribution on the basis of the actual data. Ultimately the data is downsample and would lose information. 

In [48]:
# resample on oil price data to monthly average
oil_monthly = df_oil.resample('M', on='Date').mean().reset_index(drop=False)

print(f'Dimension of resampled data: {oil_monthly.shape}')
print(f'Dimension of rig count data: {df_rig20.shape}\n')

print(oil_monthly.head())

Dimension of resampled data: (241, 10)
Dimension of rig count data: (241, 10)

        Date       High        Low       Open      Close        Volume  \
0 2000-03-31  27.582500  26.883750  27.281250  27.200000  54741.875000   
1 2000-04-30  25.948421  25.061579  25.508947  25.533158  62610.368421   
2 2000-05-31  29.203636  28.247727  28.654545  28.828637  59005.909091   
3 2000-06-30  31.910000  30.830909  31.216364  31.534546  70422.636364   
4 2000-07-31  30.198421  29.244737  29.831579  29.708948  69653.368421   

   Adj Close  year  month   adjusted  
0  27.200000  2000      3  41.008925  
1  25.533158  2000      4  38.473386  
2  28.828637  2000      5  43.388359  
3  31.534546  2000      6  47.213105  
4  29.708948  2000      7  44.376881  


## Visualization
A few functions are created to handle the repetitive and laborious plotting process. 


All regions are plotted in a stacked bar chart format for better visualization and interactivity. The alternative would have been plotting `Total_World` which would equate to the sum of all regions regardless. 

In [49]:
# function for creating stacked bar plot
def add_bar(x, y, color, yaxis='y2', opacity=0.65):
    return go.Bar(
            x=x,
            y=y,
            name=y.name,
            yaxis=yaxis,
            marker_color=color,
            opacity=opacity
                )
    
# function for creating scattered / line plot
def add_scatter(x, y, color, name):
    return go.Scatter(
            x=x, 
            y=y, 
            line=dict(color=color),
            name=name
                    )

# function for recessions areas
# all the recessions are constant so no need for additional input
def recession(fig):
    fig.update_layout(
        shapes=[

            # 2001 recession
            dict(
                type='rect',
                xref='x',
                yref='paper',
                x0='2001-03-01',
                y0=0,
                x1='2001-11-01',
                y1=1,
                fillcolor='gray',
                opacity=0.5,
                layer='above',
                line_width=0
                ),

            # 2008-2009 recession
            dict(
                type='rect',
                xref='x',
                yref='paper',
                x0='2008-01',
                y0=0,
                x1='2009-06',
                y1=1,
                fillcolor='gray',
                opacity=0.5,
                layer='above',
                line_width=0
                ),

            # 2015-2016 downturn
                dict(
                type='rect',
                xref='x',
                yref='paper',
                x0='2014-06',
                y0=0,
                x1='2016-04',
                y1=1,
                fillcolor='gray',
                opacity=0.5,
                layer='above',
                line_width=0
                    ),

            # 2020 oil war & COVID-19
                dict(
                type='rect',
                xref='x',
                yref='paper',
                x0='2020-03',
                y0=0,
                x1='2020-10',
                y1=1,
                fillcolor='gray',
                opacity=0.5,
                layer='above',
                line_width=0
                    )],

        annotations=[

            dict(
                x='2001-03',
                y=150,
                text='Recession'),

            dict(
                x='2008-01',
                y=160,
                text='Recession'),

            dict(
                x='2014-06',
                y=160,
                text='Downturn'),

            dict(
                x='2020-03',
                y=155,
                text='COVID-19'
                )
                    ])

### Information Loss

With the same data dimension (241), it would be interesting to see the difference between daily-price and monthly-average price. How much data information was kept and lost?

In [50]:
# plotting price of oil 
trace1 = add_scatter(df_oil['Date'], df_oil['adjusted'],\
           '#db1616', 'Adjusted closing oil price')


# plotting the monthly resampled oil price
trace2 = add_scatter(oil_monthly['Date'], oil_monthly['adjusted'],\
           '#000000', 'Adjusted resampled oil price')

# plotting rig count for every region
trace3 = add_bar(df_rig20['Date'], df_rig20['US'], '#41c44d') 
trace4 = add_bar(df_rig20['Date'], df_rig20['CAN'], '#81c1d6') 
trace5 = add_bar(df_rig20['Date'], df_rig20['LATAM'], '#5c81d1')
trace6 = add_bar(df_rig20['Date'], df_rig20['ME'], '#b19edb')
trace7 = add_bar(df_rig20['Date'], df_rig20['AP'], '#e87dc6')
trace8 = add_bar(df_rig20['Date'], df_rig20['EU'], '#ed3453')
trace9 = add_bar(df_rig20['Date'], df_rig20['AFR'], '#ed7e28')

# plotting all traces
fig = go.Figure(data=[trace1, trace2, trace3, trace4, trace5, 
                      trace6, trace7, trace8, trace9])

# formatting the plot
fig.update_layout(
    title = {'text': 'Price of Oil vs Regional Rig Counts',
             'x': 0.5,
             'xanchor': 'center'},
    xaxis_title = 'Date',
    yaxis_title = 'Oil price [$]',
    showlegend = True,
    legend_orientation='h',
    xaxis_tickformatstops = [
            dict(dtickrange=[86400000, 'M1'], value='%Y-%m-%d'),
            dict(dtickrange=['M1', 'M12'], value='%Y-%m'),
            dict(dtickrange=['M12', None], value='%Y')],
    yaxis1=dict(
            overlaying='y2'), 
    yaxis2=dict(
            title='Number of rigs',
            side='right'),
    barmode='stack',
    plot_bgcolor='white'
)

# add tickmarks
fig.update_xaxes(ticks='outside')
fig.update_yaxes(ticks='inside')


# add in recession areas
recession(fig)


fig.show()

The data quality was pretty well kept. A slight delay was observed on the monthly price trend compared to the daily price trend, and the local maximum and minimum were smoothened out. In this context, however, these characteristics would not make a significant difference. 

## Observations
Base on the plot, some observations were made:

- The trend of two regional rig count plots followed the general price of oil trend: `US` and `LATAM`
- These two region's rig count data lagged behind the price of oil, and not the other way around
- The other regions' rig counts did not seem to correlate with the oil price at all
- The `US` had approximately half of the world's rig count consistently
- Every region's rig count changed subtly and incrementally, even when the economy went into a recession, or recovered from the recession
- Canadian region was the only exception to the previous point

Since the active rig count is lagging behind the price of oil, cross-correlation of rig count and oil price would be examined.

In [51]:
# checking correlation with respect to each regions
df_rig20.corr()

,LATAM,EU,AFR,ME,AP,Total_Intl,CAN,US,Total_World
LATAM,1.000000,0.225203,0.172036,-0.031707,0.693130,0.636946,0.415092,0.871267,0.878540
EU,0.225203,1.000000,0.756635,0.599852,0.319162,0.690403,-0.145916,0.209831,0.307223
AFR,0.172036,0.756635,1.000000,0.862874,0.468202,0.815777,-0.227815,0.178629,0.302347
ME,-0.031707,0.599852,0.862874,1.000000,0.413528,0.731398,-0.347515,-0.016975,0.115630
AP,0.693130,0.319162,0.468202,0.413528,1.000000,0.813136,0.173134,0.678634,0.739556
Total_Intl,0.636946,0.690403,0.815777,0.731398,0.813136,1.000000,0.012404,0.580152,0.687484
CAN,0.415092,-0.145916,-0.227815,-0.347515,0.173134,0.012404,1.000000,0.494945,0.581460
US,0.871267,0.209831,0.178629,-0.016975,0.678634,0.580152,0.494945,1.000000,0.969367
Total_World,0.878540,0.307223,0.302347,0.115630,0.739556,0.687484,0.581460,0.969367,1.000000


The regions with the best cross-correlation with the `Total_World` are `LATAM` and `US`, and the `US` is also best correlated with `LATAM`. 

## Quick Interactivity
An interactive function is helpful in quickly checking the cross-correlation with different regions with respect to lag time.

In [52]:
@interact
def lag_interactive(Region=list(df_rig20.select_dtypes('number').columns),
                 Lag=(-4, 4)):
    pearson = df_rig20[Region].corr(oil_monthly['adjusted'].shift(Lag))
    print(f"Pearson R: {round(pearson, 3)}\nR Squared: {round(pearson ** 2, 3)}")

interactive(children=(Dropdown(description='Region', options=('LATAM', 'EU', 'AFR', 'ME', 'AP', 'Total_Intl', …

The interactive function confirmed, once again, that the best cross-correlation with oil price is `US` and `LATAM`. With the confirmation, the visualization can be proceeded with less distractions.

In [53]:
# t is now in units of months, so set t to be 4 months
t = 4

# loop through to store all the Pearson r values
rs = [df_rig20['US'].corr(oil_monthly['adjusted'].shift(lag)) for lag in range(-t, t + 1)]
rs1 = [df_rig20['LATAM'].corr(oil_monthly['adjusted'].shift(lag)) for lag in range(-t, t + 1)]

## Visualizing Optimal Cross-Correlations
The price of oil was shifted by 8 months - 4 months backwards and 4 months forward. The Pearson correlation coefficients were stored in `rs`. The below visualization shows where the cross-correlation is the highest. 

In [54]:
# the offset calculated in months, +1 accounts for index starting at 0
offset = np.argmax(rs) - np.ceil(len(rs) / 2) + 1
offset1 =  np.argmax(rs1) - np.ceil(len(rs1) / 2) + 1

# plotting the correlations: US and LATAM
trace1 = add_scatter([x for x in range(-int(len(rs) / 2), int(len(rs) / 2 + 1))], 
            rs, '#1f77b4', 'US')

trace2 = add_scatter([x for x in range(-int(len(rs1) / 2), int(len(rs1) / 2 + 1))], 
            rs1, '#ff7f0e', 'LATAM')


# plotting
fig = go.Figure(data=[trace1, trace2])

# formatting the plot
fig.update_layout(
    title = {'text': 'Cross-Correlations of Offsets',
             'x': 0.5,
             'xanchor': 'center'},
    xaxis_title = 'Offset [months]',
    yaxis_title = 'Pearson R'
)


# add peak indication and offset lines
fig.update_layout(
    shapes=[
        
        # center line
        dict(
            type='line',
            x0=0,
            y0=0,
            x1=0,
            y1=1,
            line=dict(
                dash='dot',
                color="black"
            )),

        # US offset line (max)
        dict(
            type='line',
            x0=offset,
            y0=0,
            x1=offset,
            y1=1,
            line=dict(
                dash='dot',
                color="#1f77b4"
            )),
        
        dict(
            type='line',
            x0=offset1,
            y0=0,
            x1=offset1,
            y1=1,
            line=dict(
                dash='dot',
                color="#ff7f0e"
            ))
        
        ],
    
    annotations=[
            dict(
                x=0,
                y=0.5,
                text='Center',
                ax=-40,
                ay=-20
                ),
        
            dict(
                    x=offset,
                    y=0.7,
                    text='US Peak Synchrony',
                    ax=-65,
                    ay=40
                ),

            dict(
                    x=offset1,
                    y=0.55,
                    text='LATAM Peak Synchrony',
                    ax=65,
                    ay=60
                )
        ])


fig.show()

## Interactive Visual
While `plotly` was a handy tool for creating interactive plots, the interactivity was limited to aesthetics. The slider, cross-correlations, and visuals could all be combined for an overall interactive experience. 

In [55]:
# function for interactivity
def update_plot(lag):
    
    # Pearson R and R squared display
    pearsonUS = df_rig20['US'].corr(oil_monthly['adjusted'].shift(lag))
    pearsonLATAM = df_rig20['LATAM'].corr(oil_monthly['adjusted'].shift(lag))
    print(f"US Pearson R: {round(pearsonUS, 3)}    LATAM Pearson R: {round(pearsonLATAM, 3)}")
    print(f"US R Squared: {round(pearsonUS ** 2, 3)}    LATAM R Squared: {round(pearsonLATAM ** 2, 3)}")
       
        
    # moving plot
    trace1 = add_scatter(oil_monthly['Date'], oil_monthly['adjusted'].shift(lag),\
           '#000000', f'Adjusted resampled oil price {lag} month lag')
    
    # bar plot for US and LATAM
    trace2 = add_bar(df_rig20['Date'], df_rig20['US'], '#41c44d') 
    trace3 = add_bar(df_rig20['Date'], df_rig20['LATAM'], '#5c81d1')
    
    # plotting
    fig = go.Figure(data=[trace1, trace2, trace3])
    
    # formatting the plot
    fig.update_layout(
        title = {'text': 'Price of Oil vs Regional Rig Counts',
                 'x': 0.5,
                 'xanchor': 'center'},
        xaxis_title = 'Date',
        yaxis_title = 'Oil price [$]',
        showlegend = True,
        legend_orientation='h',
        xaxis_tickformatstops = [
                dict(dtickrange=['M1', 'M12'], value='%Y-%m'),
                dict(dtickrange=['M12', None], value='%Y')],
        yaxis1=dict(
                overlaying='y2'), 
        yaxis2=dict(
                title='Number of rigs',
                side='right'),
        barmode='stack',
        plot_bgcolor='white'
    )
    
    # add tickmarks
    fig.update_xaxes(ticks='outside')
    fig.update_yaxes(ticks='inside')
    
    # add in recession areas
    recession(fig)
    fig.show()
    

# define slider variable
lag = IntSlider(min=-4, max=4, value=0, description='Lag [month]')
interactive(update_plot, lag=lag)

interactive(children=(IntSlider(value=0, description='Lag [month]', max=4, min=-4), Output()), _dom_classes=('…

## Analysis

* The price of oil fluctuates quite a bit despite the smoothened data - this makes sense since the oil prices are determined by the global supply and demand as well as the economy on the daily basis. 


* The best cross-correlation found was the `US` rig count lagging behind the average monthly `Oil price` by approximately 3 months. And the best cross-correlation found for `LATAM` and average monthly `Oil price` was lagging behind by approximately 4 months.  


* Overall the cross-correlation for both regions were relatively high between `2-month-offset`, `3-month-offset` and `4-month-offset`. The quality of the resampled oil price data was in `months` therefore that is the best approximation quality.



* While cross-correlation gave a good indication of the "best" offset, it did not tell the full story. 
    * It seemed like the `3-month lag` was overly shifted slightly for `US`, as seen at the beginning of the 2008 recession. However the `3-month lag` was the most appropriate shift since many of the smaller fluctuations correlated better, thus resulted in the best overall cross-correlation. On the other hand, `2-month lag` captured the major fluctuations extremely well, but not the smaller fluctuations.
    
    * It seemed like the `3-month lag` was most optimal for `LATAM` region despite `4-month lag` gave the best mathematical cross-correlation. The Pearson R value among `3-month lag` and `4-month lag` were not significant enough to conclude `4-months`, especially if the `4-month lag` seemed overly shifted. 
       
       
* `US` region had significantly more active rigs than other parts of the world, therefore it made perfect sense that `US` oil prices affected the `US` active rig count most greatly. Furthermore, `WTI oil price` is also one of the US's major oil prices - other parts of the world aren't affected by the fluctuations. Another factor to consider is that monthly average oil prices inherently has a slight lag when it was resampled from daily oil prices. 




* There are a couple of reasons that might explain why the cross-correlation is not higher, and why there is a delay:
    * Drilling a well takes a lot of planning. First, the amount of oil that would be produced would have to surpass the breakeven oil price in order to be profitable. Then begins the planning process - certified personnels would have to review and approve the logistics. The operators would have to contact third party contractors to verify the equipment availability - drill pipes, rigs, workers, drilling equipment - in order to proceed. If all goes well, drilling happens and can take upwards of months to complete the well(s). 
    
    * The process is not done after drilling has completed. The raw crude oil would have to be transported to a refinery and processed in order to produce usable products. 
    
    * For these reasons above, it's evident that if there is more sensitivity to oil price changes, then the rig count is going to change. On the other hand, if the oil price change is not significant enough, then the rig count would not be affected too much. 
    
    * Land rigs and offshore rigs are different. Land rigs tend to drill more quickly and have a faster process than offshore rigs. Lot more regulations and safety trainings have been put in place ever since [BP oil spill](https://incidentnews.noaa.gov/incident/8220), to lower the chance of such event from happening again. All of this means planning for offshore rigs takes *that* much longer.

## Conclusion

After a couple of visualization and analyzing cross-correlations, it was found that the oil price fluctuations would affect the `US` and `LATAM` active rig counts after approximately 3 months given that `US` has about 50% the world's active rig counts. The reasons behind the lag could due to the lengthy process of planning and drilling the wells, among other logistical and economical decisions. 